<h3><center>World Inequality</center></h3>
<h5><center><a href="https://github.com/Aarsh-Wankar/">Aarsh Wankar</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110003@iitgn.ac.in">23110003@iitgn.ac.in
<br><br>
<a style="text-decoration:none" href="https://github.com/jsmaskeen/">Jaskirat Singh Maskeen</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110146@iitgn.ac.in">23110146@iitgn.ac.in</a>
<br><br>
<a style="text-decoration:none" href="https://github.com/nishchaybhutoria/">Nishchay Bhutoria</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110222@iitgn.ac.in">23110222@iitgn.ac.in</a>
<br><br>
<a style="text-decoration:none" href="https://github.com/ridhamp4/">Ridham Patel</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110238@iitgn.ac.in">23110238@iitgn.ac.in</a>

</h5>

<center><a href="https://github.com/jsmaskeen/CS328-Writing-Assignment/"><img src="https://img.shields.io/badge/Github-Writing%20Assignment-brightgreen" alt="Repo"></a></center>

In [1]:
from IPython.display import HTML
import plotly.io as pio
pio.renderers.default = "notebook_connected"
HTML('''<button type="button" class="btn btn-outline-danger"  onclick="codeToggle();">Toggle Code</button>''')


#### Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import tarfile
import io
import requests
from functools import reduce

from raceplotly.plots import barplot


## Exploring the data


#### Loading and cleaning the data

In [3]:
# takes 1-3 mins
country_dataframes = {}
metadata_dataframes = {}
country_code_region_df = None
with tarfile.open(r'data\wid_all_data.tar.gz', 'r:gz') as tar:
    for member in tar.getmembers():
        if member.isfile():
            file_obj = tar.extractfile(member)
            if file_obj is None:
                continue
            filename = member.name
            if filename.endswith('.csv'):
                df = pd.read_csv(file_obj,delimiter=';')
                if 'WID_countries' in filename:
                    country_code_region_df = df
                elif 'metadata' in filename:
                    metadata_dataframes[filename.split('/')[1][:-4]] = df
                else:
                    country_dataframes[filename.split('/')[1][:-4]] = df

In [4]:
continent_codes = {
    "Africa": "QB",
    "Asia": "QD",
    "Europe": "QE",
    "North America": "QP",
    "South America": "QR",
    "Oceania": "QF"
}
continent_df = {key:country_dataframes[f'WID_data_{val}'] for key,val in continent_codes.items()}
continent_meta_data_df = {key:metadata_dataframes[f'WID_metadata_{val}'] for key,val in continent_codes.items()}


In [5]:
deleted_rows = country_code_region_df[country_code_region_df.isnull().any(axis=1)]['alpha2']
for i in deleted_rows:
    try:
        country_dataframes.pop(f"WID_data_{i}")
    except:
        pass

continent wise growth (using gdp)

In [6]:
continent_gdp = {key:continent_df[key][continent_df[key]['variable'] == 'mgdproi999'][['year','value']] for key in continent_df.keys()}
continent_gdp_df = pd.concat(
    [
        df.assign(continent=continent) 
        for continent, df in continent_gdp.items()
    ],
    ignore_index=True
)[['continent', 'year', 'value']]

my_raceplot = barplot(continent_gdp_df,
                      item_column='continent',
                      value_column='value',
                      time_column='year')

my_raceplot.plot(
    title='GDP by Continent',
    item_label='Continent',
    value_label='GDP (EUR Trillions)',
    frame_duration=300,
    date_format = '%Y',
    time_label='Year: '
)

Wealth Inequality Motion Chart
How wealth inequality has evolved with economic development, challenging the notion that economic growth naturally reduces inequality.

In [7]:
def get_countryname_from_code(code):
    return country_code_region_df[country_code_region_df['alpha2'] == code]['titlename'].to_numpy()[0]

conv_rate = requests.get('https://latest.currency-api.pages.dev/v1/currencies/eur.json').json()

def convert_to_eur(value,code):
    return value/conv_rate['eur'][code.lower()]

In [8]:
country_dataframes['WID_data_IN'].to_clipboard()

In [ ]:
data = []
for k,v in country_dataframes.items():
    v2 = v.copy()
    gdp_over_years = v2[v2['variable'] == 'agdproi992'][['year','value']]
    md = metadata_dataframes[f'WID_metadata_{k.split('_')[-1]}']
    try:
        country_currency = md[md['variable'] == 'agdproi992']['unit'].to_numpy()[0].lower()
    except:
        continue
    try:
        gdp_over_years['value'] = convert_to_eur(gdp_over_years['value'],country_currency)
    except KeyError:
        continue
    # [Net personal wealth]=[Personal non-financial assets]+[Personal financial assets]-[Personal debt]
    wealth_ineq_gini = v2[(v2['variable'] == 'ghwealj992') & (v2['percentile'] == 'p0p100')][['year','value']]
    population = v2[(v2['variable'] == 'npopuli992') & (v2['percentile'] == 'p0p100')][['year','value']]
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='year',how='inner'), [gdp_over_years, wealth_ineq_gini, population])
    merged_df.sort_values('year',inplace=True)
    if merged_df.empty:continue
    data.append([k,merged_df])

In [142]:
data.sort(key = lambda x:x[1]['value_x'][len(x[1]['value_x'])-1],reverse=True)
data2 = data[:100]
d_dcit = {k:v for k,v in data2}
common_years = set.intersection(*[set(df['year'].dropna().unique()) for df in d_dcit.values()])
d_dcit_filtered = {country: df[df['year'].isin(common_years)] for country, df in d_dcit.items()}
combined_df = pd.concat([
    df.assign(country=country) for country, df in d_dcit_filtered.items()
])
combined_df.sort_values(['country','year'],inplace=True)
combined_df.reset_index()

fig = px.scatter(
    combined_df,
    x="value_x",          # GDP per capita
    y="value_y",          # Inequality
    size="value",         # Population
    color="country",      # Color by country (or region if available)
    animation_frame="year",
    hover_name="country",
    animation_group="country",
    size_max=60,
    title="Wealth Inequality vs GDP per Capita Over Time",
    log_x=True
)

fig.update_layout(
    xaxis_title="GDP per Capita",
    yaxis_title="Wealth Inequality (Beta Coefficient)",
    template="plotly_white"
)
fig.update_yaxes(range=[-0.1,1.1])

fig.show()

The Kuznets Curve Hypothesis